# Programming Exercise

Authored by Chris Cotton, 08/14/2017

chris.j.cotton@me.com

In [1]:
import os
import sys
import re
import numpy as np
import pandas as pd
from copy import deepcopy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

## Exploration

In [164]:
##  Read data
df = pd.read_csv("/Users/chrcotto/Downloads/DS_test_merchant.csv")

##  Just for fun...
top10 = df.groupby(by = "MerchantName").count().sort_values(by = "MerchantTypeName", ascending = False).head(10)

top10

,AmountCompleted,MerchantTypeName
MerchantName,,
APL* ITUNES.COM/BILL,11147,11147
7-ELEVEN,10257,10257
SHELL Service Station,4350,4350
DOLLAR TREE,4294,4294
AMAZON MKTPLACE PMTS,3429,3429
CONOCO - UNITED PACIFIC,2855,2855
PUBLIC WORKS-PRKG METR,2543,2543
REDBOX *DVD RENTAL,2497,2497
E 470 EXPRESS TOLLS,2200,2200


In [166]:
##  I highly doubt those are correct off the bat, but we'll see what happens...

##  Check it out... what's going on?  Give me unique strings
df_unique = df.groupby(by = "MerchantName").count()
df_unique.reset_index(inplace = True)
df_unique

,MerchantName,AmountCompleted,MerchantTypeName
0,#00228 ALBERTSONS,1,1
1,#00238 ALBERTSONS,1,1
2,#00241 ALBERTSONS,1,1
3,#00242 ALBERTSONS,1,1
4,#00248 ALBERTSONS,1,1
5,#00301 ALBERTSONS,2,2
6,#00410 ALBERTSONS,2,2
7,#00486 ALBERTSONS,1,1
8,#00513 ALBERTSONS,1,1
9,#00518 ALBERTSONS,1,1


In [167]:
##  Bunch of junk... strip whitespace; keep only words and spaces
##  Remove isolated runs of digits
##  Test on unique-ified DF for faster performance

df_unique["MerchantName"] = df_unique["MerchantName"].map(lambda x: x.lstrip().rstrip())
df_unique["MerchantName"] = df_unique["MerchantName"].map(lambda x: re.sub(r'[^\w\s]', "", x))
df_unique["MerchantName"] = df_unique["MerchantName"].map(lambda x: re.sub(r'[\s\d+\s]', "", x))
df_unique

,MerchantName,AmountCompleted,MerchantTypeName
0,ALBERTSONS,1,1
1,ALBERTSONS,1,1
2,ALBERTSONS,1,1
3,ALBERTSONS,1,1
4,ALBERTSONS,1,1
5,ALBERTSONS,2,2
6,ALBERTSONS,2,2
7,ALBERTSONS,1,1
8,ALBERTSONS,1,1
9,ALBERTSONS,1,1


In [168]:
##  Apply to copy of original DF & check top 10
df_test = df.copy()
df_test["MerchantName"] = df_test["MerchantName"].map(lambda x: x.lstrip().rstrip())
df_test["MerchantName"] = df_test["MerchantName"].map(lambda x: re.sub(r'[^\w\s]', "", x))
df_test["MerchantName"] = df_test["MerchantName"].map(lambda x: re.sub(r'[\s\d+\s]', "", x))
top10_filtered = df_test.groupby(by = "MerchantName").count().sort_values(by = "AmountCompleted", ascending = False).head(10)
top10_filtered

,AmountCompleted,MerchantTypeName
MerchantName,,
MCDONALDSF,23921,23921
ELEVEN,14741,14741
SAFEWAYSTORE,14293,14293
STARBUCKSSTORE,11159,11159
APLITUNESCOMBILL,11147,11147
CHICKFILA,10600,10600
CORNERSTORE,10181,10181
TARGETT,9961,9961
COSTCOWHSE,9067,9067


In [175]:
##  7-Eleven had the 7 clipped; Target has an extra T; McDonald's has a trailing F
##  As long as we apply the same rules, we should get the same results
##  Even if the merchant names aren't perfect we can still use them as cluster centers
##  Ideally, merchant names from transactions won't change over time, but if they do,
##  and we have a procedure in place with established prior information, we can see / detect
##  any merchant name "drift"

##  Let's see how many unique vendors there are now...
df_unique_test = df_test.groupby(by = "MerchantName").count()
df_unique_test.reset_index(inplace = True)

len(df_unique_test)

##  There could be more refinements, but we've gone from ~92k merchants down to ~62k merchants, and
##  most of the merchants we have correct are big ones & have lots of transactions
##  I think we might have to do fuzzy matching at this point...regex can only go so far
##  We'd have too many rules pretty soon...would not scale well

62074

In [176]:
##  Out of curiosity...
df_test_summary = df_test.groupby(by = "MerchantName").count().sort_values(by = "AmountCompleted", ascending = False)
df_test_summary.reset_index(inplace = True)
df_test_summary.describe(np.arange(0.0, 1.0, 0.02))

,AmountCompleted,MerchantTypeName
count,62074.000000,62074.000000
mean,12.000097,12.000097
std,200.864441,200.864441
min,1.000000,1.000000
0%,1.000000,1.000000
2%,1.000000,1.000000
4%,1.000000,1.000000
6%,1.000000,1.000000
8%,1.000000,1.000000
10%,1.000000,1.000000


In [177]:
##  ...wow... > 50% of the data still has singleton merchant values...
##  > 90% of the data still has only 10 values per merchant
##  These are either unpopular merchants, or we have way more matching to do...
##  With a cursory glance, I can see Whole Foods in there...
##  We are definitely missing a ton still...
##  Time for a brutal FuzzyWuzzy matching task...

##  Some thoughts on the matter...

The issue with this problem is we don't know the labels up front...
If we had correct labels, this would be trivial...
It would be nice if vendors simply could agree on one label or title, but at least we can dream...

In the meantime, this problem boils down to an unsupervised clustering problem.
It is complicated by the fact that we don't have numerical data.
There are ways of transforming categorical data into numerical data.
There are also ways of using numerical distance metrics on categorical data.
Word2Vec takes categories and makes them continuous.
Levenshtein distance uses continuous distance metrics for categorical data.

Either way, whether you choose Word2Vec with K-Means or Levenshtein Distance,
you are still reduced to the same problem...  n x n comparisons... O(N^2)...
Levenshtein distance must compare every permutation of words to get distance
Word2Vec instantly gives vectors, but in order to find word clusters,
K-Means still must perform data-to-centroid calculations proportional to
the number of words and number of clusters; and, in this case, the number
of clusters is likely to be very high...close to a moderate fraction of the
total number of words...  (Eureka...there's a workaround to avoid O(N^2) I just realized; discussed below)

Computationally, this problem is annoying... but at least from a space-complexity
standpoint, Levenshtein distance isn't bad because we can just have a dictionary
for each word, and then a list of every word that is close to it (say, over 95%).
We still need O(N^2) time, but at least the memory footprint will be smaller.
K-Means can sometimes have memory issues, especially with clustering word vectors
and especially if those word vectors are hundreds of dimensions (like GloVe or W2V).

I am also not convinced that these words will cluster well using W2V since those
embeddings were trained on natural language and not merchant tags.
In this case, something like Levenshtein Distance will likely perform better
since it is effectively aligning strings, very similar to Needleman-Wunsch w/ DNA.
Sequences are scored not based on their embedded semantic properties per se, but
rather based on their "physical structure" (i.e., the order of the characters).

I think my approach will be to iterate through the list of tokens, each new token
that has never been seen before will become a key in a dictionary.  So, for instance,
for i = 0 in the list, that item will become the first key in the dict.
for i = 1, that item will be compared to the first item, if it is > 95% (or some %) similar,
it will be added to the dictionary as a value of the key to which it was 95% similar or will be given its own key if its too dissimilar.
This procedure will continue until all items are exhausted.

This algorithm is has linear time complexity in the best case, O(N^2) in the worst case;
and O( N log(N) ) ? I think? in average case?  (this will depend on the properties of the data itself - e.g., all identical tokens - rather than the nature of the algorithm per se)
Either way, I like that the boundary is between linear and quadratic... this is good
because if we keep a list of Merchant IDs, every new ID simply has to scan the list once
and find its "home" in the dictionary of MerchantNames.

**NOTE:**  While this algorithm will be quicker, its possible that if 2 *different* keys
"want" the same token, the first one will get it...  The alternative is to use the O(N^2)
solution and "brute force" every comparison...  The result will be a dictionary of lists,
where each *list* will then have to be compared to every other list to find the most similar
*lists of tokens*...  *then* those lists will be merged and perhaps curated if necessary.
I think I'll try the quicker method first and see what results it gives, then, if necessary,
Attempt the iterative, permutative, agglomerative approach.

**NOTE 2:**  We can boost speed and accuracy by using the existing merchant categories.
The algorithm will be applied to each category separately.
This will reduce # of comparisons and potential for mismatch.

**NOTE 3:**  After applying the initial key generation and clustering described above, this procedure could be repeated some number of times, merging the keys of the newly created dictionaries and combining their lists of values.  I'm going to try doing one of these merges; in theory, you could have the process repeat some number of times based on how much improvement each iteration yields (e.g., by some percentage of improvement; similar to a very very basic gradient descent idea).

**NOTE 4:**  Just going to do 1 initial key generation + 1 merge; rigging up a whole mechanism to intelligently do recursive merges seems like too much work for the purposes of this exercise, but some basic code is included in one of my functions to do it (its just commented out).

## Time for some code...

In [158]:
def create_category_dictionary(df_merchant_map, category):
    category_dict = {}
    merchants = df_merchant_map["MerchantName"][df_merchant_map["MerchantTypeName"] == category].tolist()

    category_dict[merchants[0]] = [merchants[0]]

    n = 0

    for merchant in merchants:
        for key in category_dict.keys():
            if fuzz.ratio(key, merchant) > 75:
                category_dict[key].append(merchant)
            else:
                category_dict[merchant] = [merchant]
        n += 1
        if n % 100 == 0:
            print str(n) + " merchants complete."

    return category_dict



def merge_keys(category_dict):

    n = 0

    for key1 in category_dict.keys():
        for key2 in category_dict.keys():
            if key1 != key2:
                try:
                    if fuzz.ratio(key1, key2) > 75:
                        category_dict[key1].extend(category_dict[key2])
                        del category_dict[key2]
                    else:
                        pass
                except:
                    pass
        n += 1
        if n % 100 == 0:
            print str(n) + " merchants complete."

    return category_dict



def recursively_merge_a_category(df_merchant_map, category):
    print "Beginning initial dictionary creation."
    category_dict = create_category_dictionary(df_merchant_map, category)
    num_keys_current = len(category_dict)
    print "Dictionay creation complete."

    print "Beginning initial key merge."
    category_dict = merge_keys(category_dict)
    num_keys_new = len(category_dict)
    print "Initial key merge complete."

    # i = 0

    # while float(num_keys_new) / float(num_keys_current) < 0.75:
        # i += 1
        # print "Beginning key merge #" + str(i)
        # category_dict = merge_keys(category_dict)
        # print "Key merge #" + str(i) + " complete."

    category_dict = {v:k for (k,vs) in category_dict.items() for v in vs}

    return category_dict




def execute_master_merge_procedure(df_merchant_map, categories):
    master_dict = {}

    for category in categories:
        print "Beginning category: " + category
        category_dict = recursively_merge_a_category(df_merchant_map, category)
        master_dict.update(category_dict)
        print "Category " + category + " complete."

    return master_dict




def read_and_apply_regex(path_to_file):
    df = pd.read_csv(path_to_file)
    df["MerchantName"] = df["MerchantName"].map(lambda x: x.lstrip().rstrip())
    df["MerchantName"] = df["MerchantName"].map(lambda x: re.sub(r'[^\w\s]', "", x))
    df["MerchantName"] = df["MerchantName"].map(lambda x: re.sub(r'[\s\d+\s]', "", x))

    return df



def prep_for_procedure(path_to_file):
    df = read_and_apply_regex(path_to_file)
    merchant_type_transactions = df.groupby(by = "MerchantTypeName").count().sort_values(by = "MerchantName", ascending = False)
    categories = merchant_type_transactions.index.tolist()
    df_merchant_map = df.groupby(by = ["MerchantName","MerchantTypeName"]).count().reset_index()[["MerchantName","MerchantTypeName"]]

    return df_merchant_map, categories




def main(path_to_file):
    df_merchant_map, categories = prep_for_procedure(path_to_file)
    master_dict = execute_master_merge_procedure(df_merchant_map, categories)
    print "Master merge procedure complete."
    return master_dict

In [113]:
input_path = "/Users/chrcotto/Downloads/DS_test_merchant.csv"
output_path = "/Users/chrcotto/Documents/saylent.txt"

In [114]:
master_dict = main(input_path)

Beginning category: FAST FOOD RESTAURANTS
Beginning initial dictionary creation.
100 merchants complete.
200 merchants complete.
300 merchants complete.
400 merchants complete.
500 merchants complete.
600 merchants complete.
700 merchants complete.
800 merchants complete.
900 merchants complete.
1000 merchants complete.
1100 merchants complete.


In [115]:
pd.DataFrame.from_dict(master_dict, orient = "index").to_csv(output_path, sep = "\t")

In [123]:
len(master_dict)

62074

In [124]:
master_dict

{'': '',
 'BROADMOORNAILSALON': 'BROADMOORNAILSALON',
 'HARPSFOODSTORE': 'HARPSFOODSTORE',
 'MEKONGMARKET': 'MEKONGMARKET',
 'MINISTRYARCHITECTUREI': 'MINISTRYARCHITECTUREI',
 'THUNDERCLOUD': 'THUNDERCLOUD',
 'TREMONTGARAGE': 'TREMONTGARAGE',
 'CHEFDISTILLEDLLC': 'CHEFDISTILLEDLLC',
 'VITAMEDMD': 'VITAMEDMD',
 'INMODSQUADVAPESHOP': 'INMODSQUADVAPESHOP',
 'GOLDPLAYERSLLC': 'GOLDPLAYERSLLC',
 'CENEXAGRIVAL': 'CENEXAGRIVAL',
 'MIMADRESRESTAURANT': 'MIMADRESRESTAURANT',
 'RENTALSVCS': 'RENTALSVCS',


### Now lets compare results from raw, regex, and fuzzy...

In [147]:
df_raw = pd.read_csv(input_path)
df_regex = read_and_apply_regex(input_path)
df_fuzzy = df_regex.copy()
df_fuzzy["MerchantName"] = df_fuzzy["MerchantName"].map(lambda x: master_dict[x])

In [148]:
top10_raw = df_raw.groupby(by = "MerchantName").count().sort_values(
    by = "AmountCompleted", ascending = False).head(10)

top10_regex = df_regex.groupby(by = "MerchantName").count().sort_values(
    by = "AmountCompleted", ascending = False).head(10)

top10_fuzzy = df_fuzzy.groupby(by = "MerchantName").count().sort_values(
    by = "AmountCompleted", ascending = False).head(10)

In [149]:
top10_raw

,AmountCompleted,MerchantTypeName
MerchantName,,
APL* ITUNES.COM/BILL,11147,11147
7-ELEVEN,10257,10257
SHELL Service Station,4350,4350
DOLLAR TREE,4294,4294
AMAZON MKTPLACE PMTS,3429,3429
CONOCO - UNITED PACIFIC,2855,2855
PUBLIC WORKS-PRKG METR,2543,2543
REDBOX *DVD RENTAL,2497,2497
E 470 EXPRESS TOLLS,2200,2200


In [150]:
top10_regex

,AmountCompleted,MerchantTypeName
MerchantName,,
MCDONALDSF,23921,23921
ELEVEN,14741,14741
SAFEWAYSTORE,14293,14293
STARBUCKSSTORE,11159,11159
APLITUNESCOMBILL,11147,11147
CHICKFILA,10600,10600
CORNERSTORE,10181,10181
TARGETT,9961,9961
COSTCOWHSE,9067,9067


In [151]:
top10_fuzzy

,AmountCompleted,MerchantTypeName
MerchantName,,
MACDONALDS,25782,25782
ELEVEN,14741,14741
SAFEWAYSTORE,14293,14293
TARGETT,12828,12828
STARBUCKSCLIFTO,11607,11607
APLITUNESCOMBILL,11147,11147
SQCHICKFILAOF,10602,10602
CORNERSTORE,10201,10201
KINGSOOPERSFUEL,9529,9529


### Some Discussion...

As one might expect, the greatest difference in results is between the raw data and either the regex-filtered or fuzzy-filtered data.

Interestingly enough, Regex & Fuzzy have 9/10 merchants in common (albeit in different rank orderings).  The only merchant they disagree on for top 10 is Shell Oil (SHELLOIL) for Regex and King Cooper's Fuel (KINGCOOPERSFUEL) for Fuzzy.

I think the main takeaway here is this: while there are some subtle improvements with a more advanced procedure, the **vast** majority of the increased accuracy is due to a literal three lines of code that takes a few seconds to execute... (i.e., applying a simple regex...)

In fact, in light of going through this whole exercise, I wonder how useful it would be to even bother with fuzzy clustering versus just thinking of more advanced and clever regexes...especially since fuzzy clustering ultimately ends up mutating the key name in unexpected ways (depends on which "seed" key the others are glommed onto; that can be random based on the order of the original data at the time of clustering while a regex is deterministic...).

At the end of the day, I would tell my client that the top 10 merchants by transaction count are the following:

In [155]:
top10_fuzzy.index.tolist()

['MACDONALDS',
 'ELEVEN',
 'SAFEWAYSTORE',
 'TARGETT',
 'STARBUCKSCLIFTO',
 'APLITUNESCOMBILL',
 'SQCHICKFILAOF',
 'CORNERSTORE',
 'KINGSOOPERSFUEL',
 'COSTCOWHSE']

...and that the the top 10 merchants by dollar amount are:

In [157]:
df_fuzzy.groupby(by = "MerchantName").sum().sort_values(by = "AmountCompleted", ascending = False).head(10)

,AmountCompleted
MerchantName,
COSTCOWHSE,776551.77
TARGETT,408202.25
THEHOMEDEPOT,363104.91
SAFEWAYSTORE,358716.99
KINGSOOPERSFUEL,227946.51
WALMART,213872.34
LOWES,177035.78
SAMSCLUB,174660.09
MACDONALDS,172842.46


### Final Thoughts

To take a step back, what I just demonstrated above are two solutions to a client ask, one more time consuming than the other.  Having worked with several clients at Digitas, every client is different.  Some want things immediately even if they're not 100% accurate.  Some don't want anything *unless* it's 100% accurate, and will happily wait until the accurate results are available.

Given that accuracy is positively associated with execution time &/or complexity, I would weigh the client's needs, and determine the best course of action.  "How fast do you need an answer?  Does it have to be perfect?"  Sometimes the answer is "Now" and "Yes" to both, in which case you have to do what you can with the time and resources that you have.  Use whatever trick in the book, borrow ideas and resources, don't re-invent the wheel, and be creative and diligent.

In this case, I was opting for accuracy, so I came up with a more intricate procedure for fuzzy-clustering key names, but a simple, 3-line-regex filter and group-by could have sufficed (and saved me a couple days of work!!).